In [38]:
from batchgenerators.utilities.file_and_folder_operations import *
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import center_of_mass, binary_dilation
from scipy.spatial import distance
from skimage.measure import label  
import pandas as pd
import os
from tqdm import tqdm

In [35]:
def compute_radius(voxel_data, center_of_mass):
    indices = np.argwhere(voxel_data == 1)
    distances = np.sqrt(np.sum((indices - center_of_mass) ** 2, axis=1))
    radius = np.max(distances)
    return radius
def cindex(folder):
    seg_file = folder+'/segmentation_sinus_r.nii.gz'
    seg = nib.load(seg_file)
    seg_data = seg.get_fdata()

    sinus_data = np.zeros(seg_data.shape)
    sinus_data[seg_data==4]=1

    # parenchyma island extract
    kid_data = np.zeros(seg_data.shape)
    kid_data[seg_data==1]=1

    kid_split = label(kid_data)
    assert(len(np.unique(kid_split))==3)

    kid_data_1 = np.zeros(seg_data.shape)
    kid_data_1[kid_split==1]=1
    kid_data_2 = np.zeros(seg_data.shape)
    kid_data_2[kid_split==2]=1

    # sinus island extract
    sinus_split = label(sinus_data)
    assert(len(np.unique(sinus_split))==3)

    sinus_data_1 = np.zeros(sinus_data.shape)
    sinus_data_1[sinus_split==1]=1
    sinus_data_2 = np.zeros(sinus_data.shape)
    sinus_data_2[sinus_split==2]=1

    # mass extract
    mass_data = np.zeros(seg_data.shape)
    mass_data[seg_data==2]=1
    #assert(len(np.unique(mass_data))==2)
    if len(np.unique(mass_data))!=2:
        return None

    # distance
    kid_data_1_center = center_of_mass(kid_data_1)
    kid_data_2_center = center_of_mass(kid_data_2)
    sinus_data_1_center = center_of_mass(sinus_data_1)
    sinus_data_2_center = center_of_mass(sinus_data_2)
    mass_center = center_of_mass(mass_data)

    dst_kd1_m = distance.euclidean(kid_data_1_center, mass_center)
    dst_kd2_m = distance.euclidean(kid_data_2_center, mass_center)
    dst_sn1_m = distance.euclidean(sinus_data_1_center, mass_center)
    dst_sn2_m = distance.euclidean(sinus_data_2_center, mass_center)

    if dst_kd1_m < dst_kd2_m:
        main_kid_data = kid_data_1.copy()
    else:
        main_kid_data = kid_data_2.copy()

    if dst_sn1_m < dst_sn2_m:
        main_sinus_data = sinus_data_1.copy()
    else:
        main_sinus_data = sinus_data_2.copy()
        
    c_data = np.zeros(seg_data.shape)
    c_data[main_kid_data==1]=1
    c_data[main_sinus_data==1]=1

    kidney_center = center_of_mass(c_data)
    radius = compute_radius(mass_data, mass_center)

    c=distance.euclidean(kidney_center, mass_center)
    c_index=c/radius
    
    return c_index

In [50]:
data_path =  '/home/ldh/1_Tindex/data/T_Index_nii_test_A/T_Index_extract_mask'
folders = subfolders(data_path)
mcindex_li=[]
pnb_li=[]
for folder in tqdm(folders):
    pnb_li.append(folder.split('/')[-1].split('_')[0])
    mcindex_li.append(cindex(folder))

100%|██████████| 21/21 [18:22<00:00, 52.49s/it]


In [51]:
data_path =  '/home/ldh/1_Tindex/data/T_Index_nii_test_A/T_Index_extract_pred'
folders = subfolders(data_path)
acindex_li=[]
for folder in tqdm(folders):
    acindex_li.append(cindex(folder))

100%|██████████| 21/21 [17:44<00:00, 50.70s/it]


In [62]:
idx=8
print(pnb_li[idx])
print(mcindex_li[idx])
print(acindex_li[idx])

542
1.1820859299410758
1.173786460184279


In [49]:
pnb_lia

['313',
 '317',
 '321',
 '323',
 '327',
 '330',
 '332',
 '334',
 '336',
 '338',
 '339',
 '341',
 '346',
 '347',
 '353',
 '361',
 '364',
 '369',
 '374',
 '376',
 '380',
 '381',
 '382',
 '384',
 '385',
 '386',
 '389',
 '391',
 '395',
 '396',
 '397',
 '398',
 '402',
 '404',
 '405',
 '406',
 '408',
 '410',
 '412',
 '413',
 '416',
 '417',
 '420',
 '430',
 '431',
 '432',
 '433',
 '434',
 '441',
 '444',
 '451',
 '457',
 '460',
 '461',
 '468',
 '469',
 '477',
 '480',
 '483',
 '486',
 '489',
 '491',
 '493',
 '498',
 '500',
 '502',
 '503',
 '504',
 '507',
 '509',
 '510',
 '513',
 '517',
 '519',
 '520',
 '522',
 '523',
 '524',
 '525',
 '529',
 '536',
 '538',
 '539',
 '540',
 '544',
 '545',
 '546',
 '547',
 '548',
 '557',
 '558',
 '559',
 '560',
 '562',
 '565']

In [2]:
def main_extract_save(folder):
    dst_sn_m, dst_mp_sn, dst_msn_sn, mp_csa, msn_csa = index_extract(folder)
    if dst_sn_m is not None:
        result_array = np.array((int(folder.split('/')[-1].split('_')[0]), dst_sn_m, dst_mp_sn, dst_msn_sn, mp_csa, msn_csa))

        save_path = folder + '/result2.npy'
        np.save(save_path, result_array)
        print(save_path)

In [3]:
def index_extract(folder):
    seg_file = folder+'/segmentation_sinus_r.nii.gz'
    seg = nib.load(seg_file)
    seg_data = seg.get_fdata()

    sinus_data = np.zeros(seg_data.shape)
    sinus_data[seg_data==4]=1
    
    # parenchyma island extract
    kid_data = np.zeros(seg_data.shape)
    kid_data[seg_data==1]=1

    kid_split = label(kid_data)
    assert(len(np.unique(kid_split))==3)

    kid_data_1 = np.zeros(seg_data.shape)
    kid_data_1[kid_split==1]=1
    kid_data_2 = np.zeros(seg_data.shape)
    kid_data_2[kid_split==2]=1

    # sinus island extract
    sinus_split = label(sinus_data)
    assert(len(np.unique(sinus_split))==3)

    sinus_data_1 = np.zeros(sinus_data.shape)
    sinus_data_1[sinus_split==1]=1
    sinus_data_2 = np.zeros(sinus_data.shape)
    sinus_data_2[sinus_split==2]=1

    # mass extract
    mass_data = np.zeros(seg_data.shape)
    mass_data[seg_data==2]=1
    #assert(len(np.unique(mass_data))==2)
    if len(np.unique(mass_data))!=2:
        return None, None, None, None, None

    # distance
    kid_data_1_center = center_of_mass(kid_data_1)
    kid_data_2_center = center_of_mass(kid_data_2)
    sinus_data_1_center = center_of_mass(sinus_data_1)
    sinus_data_2_center = center_of_mass(sinus_data_2)
    mass_center = center_of_mass(mass_data)

    dst_kd1_m = distance.euclidean(kid_data_1_center, mass_center)
    dst_kd2_m = distance.euclidean(kid_data_2_center, mass_center)
    dst_sn1_m = distance.euclidean(sinus_data_1_center, mass_center)
    dst_sn2_m = distance.euclidean(sinus_data_2_center, mass_center)

    if dst_kd1_m < dst_kd2_m:
        main_kid_data = kid_data_1.copy()
    else:
        main_kid_data = kid_data_2.copy()

    if dst_sn1_m < dst_sn2_m:
        main_sinus_data = sinus_data_1.copy()
    else:
        main_sinus_data = sinus_data_2.copy()
        
    dst_sn_m, dst_mp_sn, dst_msn_sn, mp_csa, msn_csa = distance_extract(main_sinus_data, main_kid_data, mass_data, seg.affine)
    
    return dst_sn_m, dst_mp_sn, dst_msn_sn, mp_csa, msn_csa

In [4]:
def distance_extract(main_sinus_data, main_kid_data, mass_data, affine):
    # 1. mass_center - sinus_center distance
    sinus_center = center_of_mass(main_sinus_data)
    mass_center = center_of_mass(mass_data)
    dst_sn_m = dist_3d(sinus_center, mass_center, affine)
    
    # 2. mass_parenchyma_surface - sinus_center distance (3D)
    mass_dil_data = binary_dilation(mass_data)
    x,y,z = np.where((mass_dil_data==1) & (main_kid_data==1))
    mp_ct=0
    dst_mp_sn=0
    if len(x)!=0:
        for a,b,c in zip(x,y,z):
            dst_mp_sn += 1/dist_3d(sinus_center, (a,b,c), affine)
            mp_ct+=1
            
    # 3. mass_sinus_surface - sinus_center distance (3D)
    x,y,z = np.where((mass_dil_data==1) & (main_sinus_data==1))
    msn_ct=0
    dst_msn_sn=0
    if len(x)!=0:
        for a,b,c in zip(x,y,z):
            dst_msn_sn += 1/dist_3d(sinus_center, (a,b,c), affine) 
            msn_ct+=1
            
    #print(q)
    sc=abs(affine[0][0])

    mp_csa=mp_ct*sc*sc*sc
    msn_csa=msn_ct*sc*sc*sc
            
    return dst_sn_m, dst_mp_sn, dst_msn_sn, mp_csa, msn_csa

In [5]:
def dist_3d(first, second, affine):
    # first,second : 3d point
    # affine : nib-> .affine
    dst = distance.euclidean(first, second)
    dst *= affine[0][0]
    return abs(dst)

In [57]:
#data_path =  '/home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_mask'
data_path =  '/home/ldh/1_Tindex/data/T_Index_nii'
folders = subfolders(data_path)

In [53]:
from multiprocessing import Pool
p = Pool(4)
p.map(main_extract_save, folders)
p.close()
p.join()

/home/ldh/1_Tindex/data/T_Index_nii/001/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii/023/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii/003/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii/041/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii/061/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii/005/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii/063/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii/026/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii/008/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii/064/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii/044/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii/029/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii/010/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii/048/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii/032/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii/067/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii/033/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii/016/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii/069/result2.npy
/home/ldh/1_

In [130]:
data_path =  '/home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_mask'
#data_path =  '/home/ldh/1_Tindex/data/T_Index_nii'
folders = subfolders(data_path)

In [55]:
from multiprocessing import Pool
p = Pool(8)
p.map(main_extract_save, folders)
p.close()
p.join()

/home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_mask/313_7074817_151218_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_mask/332_8305043_160804_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_mask/346_8401794_170216_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_mask/374_7505332_170913_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_mask/317_7665620_160217_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_mask/347_8386115_170117_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_mask/334_8305285_160823_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_mask/376_4303442_170904_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_mask/336_3828394_160818_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_mask/353_8416276_170222_preop/result2.npy
/home/ldh/

In [6]:
data_path =  '/home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_pred'
folders = subfolders(data_path)

In [7]:
from multiprocessing import Pool
p = Pool(8)
p.map(main_extract_save, folders)
p.close()
p.join()

/home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_pred/313_7074817_151218_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_pred/361_8444911_170405_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_pred/332_8305043_160804_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_pred/323_8261260_160602_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_pred/346_8401794_170216_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_pred/327_3398845_160725_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_pred/338_8328258_161004_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_pred/374_7505332_170913_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_pred/364_8479256_170705_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_pred/381_8562215_171121_preop/result2.npy
/home/ldh/

# Test

In [13]:
folder = '/home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_pred/313_7074817_151218_preop'
seg_file = folder+'/segmentation_sinus_r.nii.gz'
seg = nib.load(seg_file)
seg_data = seg.get_fdata()

sinus_data = np.zeros(seg_data.shape)
sinus_data[seg_data==4]=1

# parenchyma island extract
kid_data = np.zeros(seg_data.shape)
kid_data[seg_data==1]=1

kid_split = label(kid_data)
assert(len(np.unique(kid_split))==3)

kid_data_1 = np.zeros(seg_data.shape)
kid_data_1[kid_split==1]=1
kid_data_2 = np.zeros(seg_data.shape)
kid_data_2[kid_split==2]=1

# sinus island extract
sinus_split = label(sinus_data)
assert(len(np.unique(sinus_split))==3)

sinus_data_1 = np.zeros(sinus_data.shape)
sinus_data_1[sinus_split==1]=1
sinus_data_2 = np.zeros(sinus_data.shape)
sinus_data_2[sinus_split==2]=1

# mass extract
mass_data = np.zeros(seg_data.shape)
mass_data[seg_data==2]=1

In [27]:
data_path =  '/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_0'
folders=subfolders(data_path)

In [29]:
main_extract_save(folders[0])

In [13]:
data_path =  '/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_0'
folders = subfolders(data_path)
for f in folders:
    try:
        main_extract_save(f)
    except:
        print(f)

/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_0/001_2627046_211022_NP_preop_HK/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_0/002_3685594_211022_CMP_preop_KO/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_0/002_3685594_211022_NP_preop_HK/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_0/002_3685594_220325_CMP_KO/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_0/002_3685594_220325_NP_HK/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_0/003_2597321_211217_CMP_preop_KO/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_0/003_2597321_211217_NP_preop_HK/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_0/004_10506248_211220_CMP_preop_KO/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_0/004_10506248_211220_NP_preop_HK
/home/ldh

In [14]:
data_path =  '/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_1'
folders = subfolders(data_path)
for f in folders:
    try:
        main_extract_save(f)
    except:
        print(f)

/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_1/001_2627046_211022_NP_preop_HK/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_1/002_3685594_211022_CMP_preop_KO/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_1/002_3685594_211022_NP_preop_HK/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_1/002_3685594_220325_CMP_KO/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_1/002_3685594_220325_NP_HK/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_1/003_2597321_211217_CMP_preop_KO/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_1/003_2597321_211217_NP_preop_HK/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_1/004_10506248_211220_CMP_preop_KO/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_1/004_10506248_211220_NP_preop_HK
/home/ldh

In [15]:
data_path =  '/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_2'
folders = subfolders(data_path)
for f in folders:
    try:
        main_extract_save(f)
    except:
        print(f)

/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_2/001_2627046_211022_NP_preop_HK/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_2/002_3685594_211022_CMP_preop_KO/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_2/002_3685594_211022_NP_preop_HK/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_2/002_3685594_220325_CMP_KO/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_2/002_3685594_220325_NP_HK/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_2/003_2597321_211217_CMP_preop_KO/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_2/003_2597321_211217_NP_preop_HK/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_2/004_10506248_211220_CMP_preop_KO/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_2/004_10506248_211220_NP_preop_HK
/home/ldh

In [16]:
data_path =  '/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_3'
folders = subfolders(data_path)
for f in folders:
    try:
        main_extract_save(f)
    except:
        print(f)

/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_3/001_2627046_211022_NP_preop_HK/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_3/002_3685594_211022_CMP_preop_KO/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_3/002_3685594_211022_NP_preop_HK/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_3/002_3685594_220325_CMP_KO/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_3/002_3685594_220325_NP_HK/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_3/003_2597321_211217_CMP_preop_KO/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_3/003_2597321_211217_NP_preop_HK/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_3/004_10506248_211220_CMP_preop_KO/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_3/004_10506248_211220_NP_preop_HK/result.np

In [17]:
data_path =  '/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_4'
folders = subfolders(data_path)
for f in folders:
    try:
        main_extract_save(f)
    except:
        print(f)

/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_4/002_3685594_211022_CMP_preop_KO/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_4/002_3685594_211022_NP_preop_HK/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_4/002_3685594_220325_CMP_KO/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_4/002_3685594_220325_NP_HK/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_4/003_2597321_211217_CMP_preop_KO/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_4/003_2597321_211217_NP_preop_HK/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_4/004_10506248_211220_CMP_preop_KO/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_4/004_10506248_211220_NP_preop_HK/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_4/004_10506248_220603_NP_HK/result.npy
/ho

In [25]:
data_path =  '/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_mask'
folders = subfolders(data_path)
for f in folders:
    try:
        main_extract_save(f)
    except:
        print(f)

/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_mask/001_2627046_211022_CMP_preop_KO/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_mask/002_3685594_211022_CMP_preop_KO/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_mask/002_3685594_211022_NP_preop_HK/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_mask/003_2597321_211217_CMP_preop_KO/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_mask/003_2597321_211217_NP_preop_HK/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_mask/004_10506248_211220_CMP_preop_KO/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_mask/004_10506248_211220_NP_preop_HK/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_mask/005_5681816_211223_CMP_preop_KO/result.npy
/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_mask/005_5681816_211223_NP_preop_HK/result.npy
/ho

In [74]:
data_path =  '/home/ldh/1_Tindex/data/T_Index_nii_test_A/T_Index_extract_pred'
folders = subfolders(data_path)
for f in folders:
    try:
        main_extract_save(f)
    except:
        print(f)

/home/ldh/1_Tindex/data/T_Index_nii_test_A/T_Index_extract_pred/458_8794304_181227_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test_A/T_Index_extract_pred/459_3034805_181224_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test_A/T_Index_extract_pred/462_8035121_190108_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test_A/T_Index_extract_pred/463_8789193_180915_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test_A/T_Index_extract_pred/464_8279838_190122_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test_A/T_Index_extract_pred/481_8856525_190404_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test_A/T_Index_extract_pred/485_8855172_190409_preop
/home/ldh/1_Tindex/data/T_Index_nii_test_A/T_Index_extract_pred/508_5417119_190627_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test_A/T_Index_extract_pred/542_10097654_190928_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test_A/T_Index_extract_pred/573_101585253_200108_preop/result2.npy

In [75]:
data_path =  '/home/ldh/1_Tindex/data/T_Index_nii_test_A/T_Index_extract_mask'
folders = subfolders(data_path)
for f in folders:
    try:
        main_extract_save(f)
    except:
        print(f)

/home/ldh/1_Tindex/data/T_Index_nii_test_A/T_Index_extract_mask/458_8794304_181227_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test_A/T_Index_extract_mask/459_3034805_181224_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test_A/T_Index_extract_mask/462_8035121_190108_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test_A/T_Index_extract_mask/463_8789193_180915_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test_A/T_Index_extract_mask/464_8279838_190122_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test_A/T_Index_extract_mask/481_8856525_190404_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test_A/T_Index_extract_mask/485_8855172_190409_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test_A/T_Index_extract_mask/508_5417119_190627_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test_A/T_Index_extract_mask/542_10097654_190928_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test_A/T_Index_extract_mask/573_101585253_200108_preop

In [76]:
data_path =  '/home/ldh/1_Tindex/data/T_Index_nii_test_B/T_Index_extract_pred'
folders = subfolders(data_path)
for f in folders:
    try:
        main_extract_save(f)
    except:
        print(f)

/home/ldh/1_Tindex/data/T_Index_nii_test_B/T_Index_extract_pred/458_8794304_181227_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test_B/T_Index_extract_pred/459_3034805_181224_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test_B/T_Index_extract_pred/462_8035121_190108_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test_B/T_Index_extract_pred/464_8279838_190122_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test_B/T_Index_extract_pred/481_8856525_190404_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test_B/T_Index_extract_pred/485_8855172_190409_preop
/home/ldh/1_Tindex/data/T_Index_nii_test_B/T_Index_extract_pred/542_10097654_190928_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test_B/T_Index_extract_pred/573_101585253_200108_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test_B/T_Index_extract_pred/574_10166362_200130_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test_B/T_Index_extract_pred/576_10148339_191211_preop/result2.n

In [77]:
data_path =  '/home/ldh/1_Tindex/data/T_Index_nii_test_B/T_Index_extract_mask'
folders = subfolders(data_path)
for f in folders:
    try:
        main_extract_save(f)
    except:
        print(f)

/home/ldh/1_Tindex/data/T_Index_nii_test_B/T_Index_extract_mask/458_8794304_181227_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test_B/T_Index_extract_mask/459_3034805_181224_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test_B/T_Index_extract_mask/462_8035121_190108_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test_B/T_Index_extract_mask/464_8279838_190122_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test_B/T_Index_extract_mask/481_8856525_190404_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test_B/T_Index_extract_mask/485_8855172_190409_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test_B/T_Index_extract_mask/542_10097654_190928_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test_B/T_Index_extract_mask/573_101585253_200108_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test_B/T_Index_extract_mask/574_10166362_200130_preop/result2.npy
/home/ldh/1_Tindex/data/T_Index_nii_test_B/T_Index_extract_mask/576_10148339_191211_pre

In [9]:
import pandas as pd
df = pd.read_excel('/home/ldh/1_Tindex/data/PNX7Y_MR_CT_labeling_20220629.xlsx', sheet_name='Master Sheet')

In [10]:
t_index = []
data_path = '/home/ldh/1_Tindex/data/T_Index_nii'
for nb in df['연번(No.)'].values:
    result_path=None
    for d in subfolders(data_path, join=False):
        if d.startswith(str(nb).zfill(3)):
            result_path = data_path+'/'+d+'/result2.npy'
    try:
        result_array = np.load(result_path)
        t_index.append(result_array)
    except:
        t_index.append([nb,None,None,None,None,None])

In [11]:
dict_value = {
    "No" : df['연번(No.)'].values,
    "Pt_number" : df['Pt_number'].values,
    "WIT" : df['WIT'].values,
    "EBL" : df['EBL'].values,
    "Total Op time" : df['Total Op time'].values,
    "Op_date" : df['Op_date'].values,
    "Discharge_date" : df['Discharge_date'].values,
    "Preop_cr" : df['Preop_cr'].values,
    "Postop_Cr_1D" : df['Postop_Cr_1D'].values,
    "PADUA_Score" : df['PADUA_Score'].values,
    "RENAL_Score" : df['RENAL_Score'].values,
    "x_C_index" : df['x_C_index'].values,
    "y_C_index" : df['y_C_index'].values,
    "m_sn" : np.array(t_index)[:,1],
    "mp_sn" : np.array(t_index)[:,2],
    "ms_sn" : np.array(t_index)[:,3],
    "mp_csa" : np.array(t_index)[:,4],
    "ms_csa" : np.array(t_index)[:,5],
    "m_sn_ai" : np.nan,
    "mp_sn_ai" : np.nan,
    "ms_sn_ai" : np.nan,
    "mp_csa_ai" : np.nan,
    "ms_csa_ai" : np.nan,
    
}

In [12]:
new_df = pd.DataFrame(dict_value)
new_df.to_csv('/home/ldh/1_Tindex/data/T_Index_Extract_train2.csv', index=False)

In [13]:
t_index = []
t_index_ai = []
data_path = '/home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_mask'
data_ai_path = '/home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_pred'
for nb in df['연번(No.)'].values:
    result_path=None
    result_ai_path=None
    for d in subfolders(data_path, join=False):
        if d.startswith(str(nb).zfill(3)):
            result_path = data_path+'/'+d+'/result2.npy'
            result_ai_path = data_ai_path+'/'+d+'/result2.npy'
    try:
        result_array = np.load(result_path)
        t_index.append(result_array)
    except:
        t_index.append([nb,None,None,None,None,None])
        
    try:
        result_ai_array = np.load(result_ai_path)
        t_index_ai.append(result_ai_array)
    except:
        t_index_ai.append([nb,None,None,None,None,None])

In [14]:
dict_value = {
    "No" : df['연번(No.)'].values,
    "Pt_number" : df['Pt_number'].values,
    "WIT" : df['WIT'].values,
    "EBL" : df['EBL'].values,
    "Total Op time" : df['Total Op time'].values,
    "Op_date" : df['Op_date'].values,
    "Discharge_date" : df['Discharge_date'].values,
    "Preop_cr" : df['Preop_cr'].values,
    "Postop_Cr_1D" : df['Postop_Cr_1D'].values,
    "PADUA_Score" : df['PADUA_Score'].values,
    "RENAL_Score" : df['RENAL_Score'].values,
    "x_C_index" : df['x_C_index'].values,
    "y_C_index" : df['y_C_index'].values,
    "m_sn" : np.array(t_index)[:,1],
    "mp_sn" : np.array(t_index)[:,2],
    "ms_sn" : np.array(t_index)[:,3],
    "mp_csa" : np.array(t_index)[:,4],
    "ms_csa" : np.array(t_index)[:,5],
    "m_sn_ai" : np.array(t_index_ai)[:,1],
    "mp_sn_ai" : np.array(t_index_ai)[:,2],
    "ms_sn_ai" : np.array(t_index_ai)[:,3],
    "mp_csa_ai" : np.array(t_index_ai)[:,4],
    "ms_csa_ai" : np.array(t_index_ai)[:,5],
}

In [15]:
new_df = pd.DataFrame(dict_value)
new_df.to_csv('/home/ldh/1_Tindex/data/T_Index_Extract_test2.csv', index=False)

In [16]:
t_index = []
t_index_ai = []
data_path = '/home/ldh/1_Tindex/data/T_Index_nii_test_B/T_Index_extract_mask'
data_ai_path = '/home/ldh/1_Tindex/data/T_Index_nii_test_B/T_Index_extract_pred'
for nb in df['연번(No.)'].values:
    result_path=None
    result_ai_path=None
    for d in subfolders(data_path, join=False):
        if d.startswith(str(nb).zfill(3)):
            result_path = data_path+'/'+d+'/result2.npy'
            result_ai_path = data_ai_path+'/'+d+'/result2.npy'
    try:
        result_array = np.load(result_path)
        t_index.append(result_array)
    except:
        t_index.append([nb,None,None,None,None,None])
        
    try:
        result_ai_array = np.load(result_ai_path)
        t_index_ai.append(result_ai_array)
    except:
        t_index_ai.append([nb,None,None,None,None,None])

In [17]:
dict_value = {
    "No" : df['연번(No.)'].values,
    "Pt_number" : df['Pt_number'].values,
    "WIT" : df['WIT'].values,
    "EBL" : df['EBL'].values,
    "Total Op time" : df['Total Op time'].values,
    "Op_date" : df['Op_date'].values,
    "Discharge_date" : df['Discharge_date'].values,
    "Preop_cr" : df['Preop_cr'].values,
    "Postop_Cr_1D" : df['Postop_Cr_1D'].values,
    "PADUA_Score" : df['PADUA_Score'].values,
    "RENAL_Score" : df['RENAL_Score'].values,
    "x_C_index" : df['x_C_index'].values,
    "y_C_index" : df['y_C_index'].values,
    "m_sn" : np.array(t_index)[:,1],
    "mp_sn" : np.array(t_index)[:,2],
    "ms_sn" : np.array(t_index)[:,3],
    "mp_csa" : np.array(t_index)[:,4],
    "ms_csa" : np.array(t_index)[:,5],
    "m_sn_ai" : np.array(t_index_ai)[:,1],
    "mp_sn_ai" : np.array(t_index_ai)[:,2],
    "ms_sn_ai" : np.array(t_index_ai)[:,3],
    "mp_csa_ai" : np.array(t_index_ai)[:,4],
    "ms_csa_ai" : np.array(t_index_ai)[:,5],
}

In [18]:
new_df = pd.DataFrame(dict_value)
new_df.to_csv('/home/ldh/1_Tindex/data/T_Index_Extract_testB.csv', index=False)